In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import torch
from torch.distributions.normal import Normal

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Initial State
        self.s_0 = nn.Parameter(torch.tensor([5.525, 5.1, 5.1,5.1], requires_grad=True)).requires_grad_(True)
        
        self.T = nn.Parameter(torch.tensor(torch.rand(4,4), requires_grad=True)).requires_grad_(True)
        self.mu1 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
        
        self.mu2 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
        
        self.std1 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
#         self.std2 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
        

        
        
        

    def forward(self, x,t,pred):

        x = F.softmax((x.clone()),dim=0)
        l=0.0
        std1=torch.exp(self.std1)
        

        x_temp=x.clone()
            
            
            
            
        dist=Normal(self.mu1[0]+self.mu2[0]*0.0, std1[0])
        
        o_1 = torch.exp(dist.log_prob(pred[0]))

        x_temp[0]=o_1*x[0].clone()

        dist=Normal(self.mu1[1]+self.mu2[1]*0.0, std1[1])

        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[1]=o_1.clone()*x[1].clone()

        dist=Normal(self.mu1[2]+self.mu2[2]*0.0, std1[2])
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[2]=o_1*x[2].clone()
        
        dist=Normal(self.mu1[3]+self.mu2[3]*0.0, std1[3])
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[3]=o_1*x[3].clone()
        
#         print(x_temp)
        


        x_temp=x_temp.clip(1e-6,1)
        
        x=x_temp.clone()/torch.sum(x_temp.clone())
        l+=torch.log(torch.sum(x_temp.clone()))# -0.01*state
        
        for i in range(1,t): 

            
            x=torch.matmul(x,F.softmax(self.T,dim=1))
            x_temp=x.clone()
            

            x_temp1=x.clone()
            dist=Normal(self.mu1[0]+self.mu2[0]*i, std1[0])
        
            o_1 = torch.exp(dist.log_prob(pred[i]))

            x_temp[0]=o_1*x[0].clone()

            dist=Normal(self.mu1[1]+self.mu2[1]*i, std1[1])

            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[1]=o_1.clone()*x[1].clone()

            dist=Normal(self.mu1[2]+self.mu2[2]*i, std1[2])
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[2]=o_1*x[2].clone()

            dist=Normal(self.mu1[3]+self.mu2[3]*i, std1[3])
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[3]=o_1*x[3].clone()

            
            
            
#             print(x_temp)
#             print(pred[i])
            x_temp=x_temp.clip(1e-6,1)

            x=x_temp.clone()/torch.sum(x_temp.clone())
#             print(x)
#             print("-----------------------------")
            l+=torch.log(torch.sum(x_temp.clone()))
        return l
    
    def transition(self,x):
        x=torch.matmul(x.to(device),F.softmax(self.T,dim=1))
        x_temp=x.clone()
        return x
    def Observation(self,x):

#         o_1= self.emission((x))
#         o_1[1]=torch.exp(o_1[1])

        return mu
    
    def simulate(self):
         o=[]
         x1 = 0
         for i in range(0,1000):
             print(x1)
            
            
            
            
             if x1==0:
                x=torch.tensor([1.0,0.0,0]).to(device)
             if x1==1:
                x=torch.tensor([0, 1.0,0.0]).to(device)
             if x1==2:
                x=torch.tensor([0.0,0.0,1]).to(device)
             o_1= self.emission((x)) 
             o_1=o_1/torch.sum(o_1)

             o.append(torch.distributions.Categorical(o_1.to(device)).sample())
             if x1==2:
                
                return o
             x = self.transition((x))
             x1=(torch.distributions.Categorical(x.to(device)).sample()).item()



In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'

cuda:0


In [5]:
import torch

def generate_random_transition_matrix(size):
    # Generate a random matrix
    random_matrix = torch.rand(size, size)
    
    # Increase weights on the diagonal elements
    diagonal_weights = 8.0
    random_matrix = random_matrix + torch.diag(torch.diagonal(random_matrix) * (diagonal_weights - 1))

    # Normalize each row to ensure the probabilities sum to 1
    random_matrix = random_matrix / random_matrix.sum(dim=1, keepdim=True)

    # Round the elements to 2 decimal places
    random_matrix = torch.round(random_matrix * 100) / 100.0

    # Ensure no value is zero
    random_matrix = torch.clamp(random_matrix, min=0.01)

    # Re-normalize to ensure it's still a valid transition matrix
    random_matrix = random_matrix / random_matrix.sum(dim=1, keepdim=True)

    return random_matrix

# Set the size of the random matrix (change this based on your needs)
matrix_size = 4

# Generate a random transition matrix with increased weights on the diagonal,
# rounded to 2 decimal places, ensuring no value is zero,
# and re-normalized to ensure it's still a valid transition matrix
random_transition_matrix = generate_random_transition_matrix(matrix_size)

# Print the modified, rounded, non-zero, and re-normalized transition matrix
print("Modified, Rounded, Non-zero, and Re-normalized Random Transition Matrix:")
print(random_transition_matrix)


Modified, Rounded, Non-zero, and Re-normalized Random Transition Matrix:
tensor([[0.8788, 0.0404, 0.0303, 0.0505],
        [0.1400, 0.4600, 0.3300, 0.0700],
        [0.1200, 0.1200, 0.6800, 0.0800],
        [0.1000, 0.2000, 0.0600, 0.6400]])


In [6]:

T_matrix=torch.rand(3, 3).to(device)
T_matrix=torch.tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]], device=device)

# T_matrix[1,1]=8

# T_matrix[2,2]=8

# T_matrix[0,0]=8

T_matrix=T_matrix/torch.sum(T_matrix, 1).reshape(4,1)


s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)

O_matrix_mean=torch.tensor([1.5,2.5,3.5,5.5]).to(device)
O_matrix_std=torch.tensor([1.1,1.2,1.3,1.4]).to(device)


O_matrix_mean1=torch.tensor([0.1,0.2,0.3,0.4]).to(device)



print(T_matrix)

print(s_0)

tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]])
tensor([1., 0., 0.])


In [7]:
def simulate_HMM(T_matrix,O_matrix,x,t):
    x=0
    
    
    t=0
    o=[]
#     o_1=torch.matmul(x.to(device),O_matrix.to(device))
# #         print(o_1)
    o.append(Normal(O_matrix_mean[x],O_matrix_std[x]).sample().to(device))
    t=1
    for i in range(1,1000):
#         print(i)
        x=(torch.distributions.Categorical(T_matrix[x].to(device)).sample())
#         print(x)
        
        
        t+=1
        o.append(Normal(O_matrix_mean[x]+(O_matrix_mean1[x]*torch.tensor(i)),O_matrix_std[x]).sample().to(device))
        t1=t
           
        if (t==50) :
            break
    
    
    
    o=torch.tensor(o).to(device)
    return o,t,t1



In [8]:
o=simulate_HMM(T_matrix,O_matrix_mean,torch.tensor([1.0,0,0.0]),1000)
o

(tensor([ 1.3446,  1.5858,  4.0008,  5.9071,  2.9921,  7.8541,  6.0860,  9.5073,
          7.9439,  7.1376,  6.4760,  6.2427,  8.9799,  7.7525, 11.6505,  5.2033,
          5.5900,  5.0743,  8.8731,  8.3738,  9.5643,  7.1680,  8.0886,  7.3450,
          5.8815,  4.9184,  2.8661,  3.8686,  4.0872, 17.4782, 16.9078,  7.7811,
          9.7506,  4.3030,  6.5767, 12.6362,  9.6165,  8.5897,  7.4625, 10.4829,
          3.1507, 10.7668, 10.4365, 18.1451, 11.3472, 10.9129, 11.2973,  6.6965,
          7.3936, 11.9892]),
 50,
 50)

In [9]:
# def distribution_initalize(o):
#     s=torch.zeros(10)
#     for i in range(0,10):
#         s[i]=torch.sum(o==i)
        
#     return s#/torch.sum(s)
# def distribution_update(dist,o):

#     return dist + distribution_initalize(o)


In [10]:
# f=distribution_initalize(o[0])

def Expectation_diff(o_1):
    return torch.sum(o_1[0])


def abs_diff(e):
    k=0
    for i,e1 in enumerate(e):
        for j,e2 in enumerate(e):
            if j==i+1:
                k+=torch.abs(e2-e1)
    return k


In [11]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import time

start_time = time.time()  

for batch_size in [5]:
    trial =0 
    while trial < 5:
        t=50
        # batch_size=20
        try: 
            pred1=[]
            for i in range(0,batch_size):
                pred1.append(simulate_HMM(T_matrix,O_matrix_mean,s_0,t))
            import torch.optim as optim
            net = Net()
            net.to(device)
            optimizer = optim.Adam(net.parameters(), lr=.01)
            scheduler =lr_scheduler.LinearLR(optimizer,start_factor=1, end_factor=0.001, total_iters=75)
            net.train()
    
            running_loss = 0.0
            o,t,t__1=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
            # dist=distribution_initalize(o)
            for epoch in range(100):  # loop over the dataset multiple times
    
    
    
    
                # zero the parameter gradients
                optimizer.zero_grad()
    
                # forward + backward + optimize
    
    
    
                loss=0.0
    
                for i in range(0,batch_size):
                    optimizer.zero_grad()
                    loss=0.0
    
            #         o,t=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
                    o,t,t_11=pred1[i]
                    pred=torch.zeros((1,t), dtype=torch.int32)
                    pred=o
            #         dist=distribution_update(dist,o)
                    loss-=net(net.s_0,t,pred.to(device))
    
                    l1_lambda = 0.001  # adjust the L1 regularization parameter as needed
                    l1_reg = torch.tensor(0.).to(device)
                    for param in net.parameters():
                        l1_reg += torch.norm(param, 1).to(device) 
                    f=0.0
                #     print(loss)
                    loss=loss
                    loss.backward()
                    optimizer.step()
                scheduler.step()
    
            torch.save(pred1, "data_normal\ " + str(trial)+" _ "+str(batch_size)+ "_data.pt")
            torch.save(net.state_dict(),  "Model_normal\ " + str(trial)+" _ "+str(batch_size)+ "_model.pth")
            trial+=1
        except:
            print(f'[{trial + 1}] loss: {loss/t}')
            
            
            pass
end_time = time.time()  # Capture end time after the loop completes

print(f"Total execution time: {end_time - start_time} seconds")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8752\777693682.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.T = nn.Parameter(torch.tensor(torch.rand(4,4), requires_grad=True)).requires_grad_(True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8752\777693682.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mu1 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8752\777693682.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor)

Total execution time: 320.3298280239105 seconds
